In [ ]:
from grid.model.perception.depth.metric3d import Metric3D
depth_metric3d_0 = Metric3D()

In [ ]:
# run this cell everytime you want to compute the point cloud from rgb image
# We use Metric3d model along with utilities built in airgen to compute and
# visualise the point cloud in rerun. We also get the depth map here to see
# how it looks

import numpy as np
import airgen
from PIL import Image
import rerun as rr
from airgen.utils.vision import depth2pointcloud

client = airgen.VehicleClient()

images = client.getImages("front_center", [airgen.ImageType.Scene, airgen.ImageType.DepthPerspective])
capture = {
        "front_rgb": images[0],
        "front_depth": images[1],
    }

for camera_name, camera_data in capture.items():
    if camera_name == "front_depth":
        # compute point cloud and visualize it in rerun
        # remove points that are too far away
        mask = np.where(camera_data[0] < 1000.0, 1, 0).astype(np.uint8)
        point_cloud = depth2pointcloud(
            depth=camera_data[0], camera_param=camera_data[1], mask=mask
        )
        print(point_cloud.shape)
        print(camera_data[0].shape)
        rr.log('grid/point_cloud', rr.Points3D(positions = point_cloud))
        rr.log('grid/depth_ground_truth', rr.DepthImage(camera_data[0]))
    else:
        print(camera_data[0].shape)
        rr.log('grid/rgb', rr.Image(camera_data[0]))
        depth_map_pred = depth_metric3d_0.run(camera_data[0])
        print("Predicted Depth map shape:", depth_map_pred.shape)
        # reshape to be compliant to the depth2pointcloud api
        depth_map_3d_reshape = depth_map_pred.reshape((256, 256, 1))
        mask = np.where(depth_map_3d_reshape < 150.0, 1, 0).astype(np.uint8)
        point_cloud = depth2pointcloud(
            depth=depth_map_3d_reshape, camera_param=camera_data[1], mask = mask
        )
        print("Predicted Depth map after reshape:", depth_map_3d_reshape.shape)
        rr.log('grid/point_cloud_predicted', rr.Points3D(positions = point_cloud))
        rr.log('grid/depth_predicted', rr.DepthImage(depth_map_3d_reshape))